In [46]:
import torch
import time
from torch_geometric.utils import to_dense_adj

In [47]:
nse = 5
dims = (5, 5, 2, 2)
I = torch.cat([torch.randint(0, dims[0], size=(nse,)),
               torch.randint(0, dims[1], size=(nse,))], 0).reshape(2, nse)
V = torch.randn(nse, dims[2], dims[3])
S = torch.sparse_coo_tensor(I, V, dims).coalesce()
D = torch.randn(dims)
D.sparse_mask(S)

tensor(indices=tensor([[0, 1, 4, 4, 4],
                       [1, 0, 0, 2, 3]]),
       values=tensor([[[ 9.7430e-01,  4.6778e-01],
                       [-1.5608e+00, -1.9542e+00]],

                      [[ 9.3249e-02, -3.4535e-01],
                       [-1.1144e+00, -1.9609e+00]],

                      [[ 7.0071e-01, -8.1074e-01],
                       [-4.6243e-01, -2.6070e-01]],

                      [[-2.5342e+00,  2.1944e+00],
                       [-9.2033e-01, -7.8521e-01]],

                      [[ 8.1250e-04, -5.4019e-01],
                       [ 1.2566e+00, -1.1428e+00]]]),
       size=(5, 5, 2, 2), nnz=5, layout=torch.sparse_coo)

In [52]:
edge_index = torch.tensor([[0,0,1,1,2,2],[1,2,3,4,4,5]], dtype=torch.long)  # 2 x E
x = torch.tensor([[1], [1], [1], [1], [1], [1]], dtype=torch.float)  # N x emb(in)
print("Original Edge_index: \n", edge_index)

def one_step_sparse(edge_index, x):
    edge_weight = None
    if x>1: 
        edge_weight = torch.ones(edge_index.size(1))
        size = torch.Size([edge_index.size(1), edge_index.size(1)] + list(edge_weight.size())[1:])
        adj = torch.sparse.FloatTensor(edge_index, edge_weight, size = size)
        #print(adj.to_dense())
        #adj = to_dense_adj(edge_index)[0]
        for k in range(2, x+1):
            adj_k = (torch.sparse.mm(adj, adj) - adj)
            _ind = adj_k._indices()
            _val = 1/k * (adj_k._values()>0)*1

            size = torch.Size([6, 6] + list(_val.size())[1:])
            adj = torch.sparse.FloatTensor(_ind, _val, size = size)
            #print("Adjazenz_ adj)
            #print("Adjazenz_dense", adj.to_dense())
            adj = adj+adj_k

        #print(adj.to_dense())
        return adj
            
    return edge_index

avg_passed = 0
n= 0
for i in range(1):   
    start = time.time_ns()
    hop2 = one_step_sparse(edge_index, 2)
    end = time.time_ns()
    passed = 1 * (end-start)
    if passed>0:
        n+=1
    avg_passed += passed

print()
print("Hop2-Sparse: \n", hop2)
print()
print("Avg. time: \n", avg_passed/n)

Original Edge_index: 
 tensor([[0, 0, 1, 1, 2, 2],
        [1, 2, 3, 4, 4, 5]])
tensor([[ 0.0000, -1.0000, -1.0000,  1.5000,  2.5000,  1.5000],
        [ 0.0000,  0.0000,  0.0000, -1.0000, -1.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000, -1.0000, -1.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]])

Hop2-Sparse: 
 tensor(indices=tensor([[0, 0, 0, 0, 0, 1, 1, 2, 2],
                       [1, 2, 5, 4, 3, 3, 4, 4, 5]]),
       values=tensor([-1.0000, -1.0000,  1.5000,  2.5000,  1.5000, -1.0000,
                      -1.0000, -1.0000, -1.0000]),
       size=(6, 6), nnz=9, layout=torch.sparse_coo)

Avg. time: 
 995700.0


In [42]:
edge_index = torch.tensor([[0,0,1,1,2,2],[1,2,3,4,4,5]], dtype=torch.long)  # 2 x E
x = torch.tensor([[1], [1], [1], [1], [1], [1]], dtype=torch.float)  # N x emb(in)
print("Original Edge_index: \n", edge_index)

def one_step(edge_index, x):
    edge_weight = None
    if x>1: 
        edge_weight = torch.ones(edge_index.size(1))
        adj = to_dense_adj(edge_index)[0]
        for k in range(2,x+1):
            adj_k = (torch.linalg.matrix_power(adj, k)>0)*1 - adj
            print(adj_k)
            edge_index_k = (adj_k > 0).nonzero().t()
            edge_weight_k = 1/k * torch.ones(edge_index_k.size(1))
            edge_index = torch.cat((edge_index, edge_index_k), dim=1)
            edge_weight = torch.cat((edge_weight, edge_weight_k), dim=0)

    return edge_index, edge_weight

avg_passed = 0
n= 0
for i in range(1):   
    start = time.time_ns()
    hop2, w2 = one_step(edge_index, 2)
    end = time.time_ns()
    passed = 1 * (end-start)
    if passed>0:
        n+=1
    avg_passed += passed

print()
print("Hop2-Edge-Index:\n", hop2)
print("Edge-Weights:\n", w2)
print()
print("Avg. time:", avg_passed/n)

 0.,  0.,  0.]])
tensor([[ 0., -1., -1.,  1.,  1.,  1.],
        [ 0.,  0.,  0., -1., -1.,  0.],
        [ 0.,  0.,  0.,  0., -1., -1.],
        [ 0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.]])
tensor([[ 0., -1., -1.,  1.,  1.,  1.],
        [ 0.,  0.,  0., -1., -1.,  0.],
        [ 0.,  0.,  0.,  0., -1., -1.],
        [ 0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.]])
tensor([[ 0., -1., -1.,  1.,  1.,  1.],
        [ 0.,  0.,  0., -1., -1.,  0.],
        [ 0.,  0.,  0.,  0., -1., -1.],
        [ 0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.]])
tensor([[ 0., -1., -1.,  1.,  1.,  1.],
        [ 0.,  0.,  0., -1., -1.,  0.],
        [ 0.,  0.,  0.,  0., -1., -1.],
        [ 0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.]])
tensor([[ 0., -1., 